# Tests for `load_data` 
This notebook serves to address [this](https://github.com/ciemss/pyciemss/issues/434) Github issue.

The new interface for calibrate requires csv files, but there are lots of ways that csv files can fail to provide information in the right format. Test the most common failure modes, such as:

- missing data
- incorrectly typed columns
- mislabeled columns
- header columns have one fewer column than data
- alignment issues
- Escaping commas
- Na, NaN, None, '',,

All of these issues will make it difficult to convert a dataframe to a correctly typed tensor.

### Missing data

In [14]:
dataset1 = "/Users/altu809/Projects/pyciemss/docs/source/sa-testing-notebooks/SIR_missing_data.csv"

import csv

# Specify the path to your CSV file
# csv_file_path = 'path/to/your/file.csv'

# Open the CSV file and create a CSV reader
with open(dataset1, 'r') as file:
    csv_reader = csv.reader(file)

    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Print each row
        print(row)


# TODO: write tests for missing data

['Timestamp', 'Infected', 'Hospitalized']
['1.1', '15.0', '0.1']
['2.2', '18.0', '']
['3.3', '20.0', '2.2']


### Incorrectly typed columns

In [9]:
# Get the path of the Jupyter Notebook
notebook_path = os.path.abspath('__file__')

# Print the path
print(notebook_path)

/Users/altu809/Projects/pyciemss/docs/source/sa-testing-notebooks/__file__


### Mislabeled columns

In [11]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Specify the relative path to your file
relative_path = 'path/to/your/file.txt'

# Construct the full path
file_path = os.path.join(current_directory, relative_path)

# Print the full path
print(file_path)


/Users/altu809/Projects/pyciemss/docs/source/sa-testing-notebooks/path/to/your/file.txt


### Header columns have one fewer column than data

### Other alignment issues

### Escaping commas

### Na, NaN, None, empty string